In [1]:
import time
import multiprocessing

In [5]:
# Unix, Linux, Windows의 Task Management(운영체제의 5대 요소중 하나)
# 이를 제어하는 방식이 다르기 때문에 multiprocessing의 경우
# Linux가 아닌 경우 동작에 문제가 발생할 수 있어보임

# ps -ef
# 윈도우의 Ctrl + Alt + Delete와 같은 기능을 수행함

# 사실 이와 같은 부분 때문에 네트워크 장비들(스위치, 라우터) 계열이 전부 리눅스를 사용함
# 속도 측면의 이슈가 큼(AWS - 리눅스)
def count_down(x):
    while True:
        if x == 0:
            break
            
        print("카운트: %d" % x)
        x -= 1
        time.sleep(1)
        
p = multiprocessing.Process(target=count_down, args=(5, ))

print(p)

p.start()

<Process name='Process-3' parent=1220 initial>


In [6]:
# 윈도우, 맥은 멀티 프로세스 라이브러리가 사용이 불가능함
# 그러므로 바로 스레드 라이브러리를 적용하도록 한다.
import threading

# Java에서 Thread를 사용했었는데 여기 중요한 개념이 몇 개 있음
# 1) Critical Section 은 무엇인가 ?
#    여러 프로세스나 스레드가 동시다발적으로 접근(write가 문제)할 수 있는 메모리 공간(변수 및 객체)
# 2) 동기화 메커니즘(Mutex, Semaphore, Spinlock)을 사용하는 이유는 무엇인가 ?
# ex) 회식 이후 친구가 계산했고 회식비 100만원 반땅하기로 했음
#     1] 회식 이후 바로 송금
#     2] 회식후 지하철(택시등등)에서 송금
#     3] 회식후 집에가서 송금
#     4] 회식후 몇 일 후 송금
#     5] 회식후 몇 주 후 송금
#     6] 회식후 몇 달 후 송금
#     7] 회식후 몇 년 후 송금
#     8] 회식후 장례식에서 송금

# A 스레드
# b = 1; c = 2
# a = b + c     a = 3
# b = c + d   <--- 실행
# print(b)    <--- 원래는 2 + d가 되어야 할 것이 4 + d가 되었음

# B 스레드
# b = 3; c = 4
# f = b + c    f = 7
# c = f + a   <--- 실행
# print(c)

In [9]:
def print_hello():
    print("Hello")
    time.sleep(1)
    print("Finish Sleep")

if __name__ == '__main__':
    # 현재 시각을 계측함(start)
    start = time.perf_counter()
    threads = []
    
    # 5개의 스레드 객체를 생성
    for _ in range(5):
        t = threading.Thread(target = print_hello)
        # start()를 해야지만 스레드가 구동 가능해짐
        t.start()
        threads.append(t)
        
    # 생성한 스레드 5개를 모두 구동시킴(Hello 출력)
    # 이후 1초 쉬고 Finish Sleep 출력
    for thread in threads:
        # 실제 구동은 join()에서 한다고 보면 됨
        thread.join()
        
    # 현재 시각을 계측(finish)
    finish = time.perf_counter()
    
    # 끝 - 시작 = 동작하는데 걸린 시간
    print("Finished: ", str(round(finish - start, 2)) + "초")

Hello
Hello
Hello
Hello
Hello
Finish SleepFinish Sleep

Finish Sleep
Finish Sleep
Finish Sleep
Finished:  1.04초


In [24]:
# Critical Section에서 데이터 무결성이 지켜지지 않아 발생하는 문제
import threading

x = 0

def increment_global():
    global x
    x += 1
    
def thread_task():
    # _는 아무것도 받지 않고 그냥 넘기고 싶을 경우 사용함
    for _ in range(1000000):
        increment_global()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_task)
    t2 = threading.Thread(target = thread_task)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))

0 번째 반복 이후 x = 1480484
1 번째 반복 이후 x = 1682308
2 번째 반복 이후 x = 1758563
3 번째 반복 이후 x = 1601512
4 번째 반복 이후 x = 1872100
5 번째 반복 이후 x = 1870972
6 번째 반복 이후 x = 1863240
7 번째 반복 이후 x = 1732919
8 번째 반복 이후 x = 1786356
9 번째 반복 이후 x = 1813023


In [29]:
# Critical Section에 동기화 메커니즘을 적용하여 데이터 무결성을 보장해보자!
import threading

lock = threading.Lock()

x = 0

def increment_global():
    global x
    x += 1
    
def thread_task():
    # _는 아무것도 받지 않고 그냥 넘기고 싶을 경우 사용함
    for _ in range(1000000):
        lock.acquire()
        increment_global()
        lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_task)
    t2 = threading.Thread(target = thread_task)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))

0 번째 반복 이후 x = 2000000
1 번째 반복 이후 x = 2000000
2 번째 반복 이후 x = 2000000
3 번째 반복 이후 x = 2000000
4 번째 반복 이후 x = 2000000
5 번째 반복 이후 x = 2000000
6 번째 반복 이후 x = 2000000
7 번째 반복 이후 x = 2000000
8 번째 반복 이후 x = 2000000
9 번째 반복 이후 x = 2000000


In [30]:
# 동기화 메커니즘은 Thread에 Context Switching(컨텍스트 스위칭)을 유발한다.
# Memory Hierarchy(메모리 계층구조) 관점에서 메모리에 자주 접근하는 행위는 좋지 못하다.
import threading

lock = threading.Lock()

x = 0

def increment_global():
    global x
    x += 1
    
def thread_task():
    lock.acquire()
    
    # _는 아무것도 받지 않고 그냥 넘기고 싶을 경우 사용함
    for _ in range(1000000):    
        increment_global()
        
    lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_task)
    t2 = threading.Thread(target = thread_task)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))

0 번째 반복 이후 x = 2000000
1 번째 반복 이후 x = 2000000
2 번째 반복 이후 x = 2000000
3 번째 반복 이후 x = 2000000
4 번째 반복 이후 x = 2000000
5 번째 반복 이후 x = 2000000
6 번째 반복 이후 x = 2000000
7 번째 반복 이후 x = 2000000
8 번째 반복 이후 x = 2000000
9 번째 반복 이후 x = 2000000


In [ ]:
# 1. 연산속도 보다 우선시 해야할 것은 데이터의 무결성
#    최소한 1번을 지켜서 작업을 해주도록 한다.
# 2. 데이터 무결성이 보장된다는 가정하에 속도까지 올려주면 금상첨화
# 3. 이런 속도를 올리는 구간에서는 기본기(이론)가 큰 영향을 끼치게 됨
# 일반적으로는 1번이 가장 중요, 2, 3번은 곁다리

# 논외로 취급하고 <<<
# 4. 짬이 좀 더 차면 스레드를 공유되지 않게 사용해서 연산을 분할하는 방식을 취해도 됨
# ex) 배열 인덱스 0 에는 A 스레드가 계산한 결과
#     배열 인덱스 1 에는 B 스레드가 계산한 결과
#     배열 인덱스 2 에는 C 스레드가 계산한 결과
#     각 배열의 값을 합산하여 최종 결과를 얻음
#     원래 걸릴 시간의 1/3 만 활용하며 동기화 없이 안정된 결과를 얻을 수 있음 <<<